In [1]:
import pandas as pd
import sqlite3
from file_extraction import *
from to_dataframe import *
from clean_dfs import *

In [2]:
files_dict, academy_csvs_file_names = extract()
dataframes = convert(files_dict, academy_csvs_file_names)

In [3]:
df_academy_csv = dataframes['academy_csv']
df_academy_csv = clean_academy_csv(df_academy_csv)

In [4]:
df_csv = dataframes['csv']
df_csv = clean_talent_csv(df_csv)

In [5]:
df_json = dataframes['json']
df_json = clean_talent_json(df_json)

In [6]:
df_txt = dataframes['txt']
df_txt = clean_talent_txt(df_txt)

In [7]:
# makes an address table dataframe 
def generate_address_df(df):
    addresses = list()
    # for each row in the talent csv files, add each unique address, city and postcode to a list
    for index, row in df.iterrows():
                address_list = list()
                address_list.append(row["address"])
                address_list.append(row["city"])
                address_list.append(row["postcode"])
                if address_list not in addresses:
                    addresses.append(address_list)
                    
    address_table = pd.DataFrame(addresses, columns=["address", "city", "postcode"])
    address_table.index = address_table.index + 1
    address_table['address_id'] = address_table.index
    return address_table

In [8]:
address_table = generate_address_df(df_csv)

In [58]:
def get_person_ids(applicants):
    applicants = applicants.reset_index().reset_index().drop(['index', 'id'], axis=1).rename(columns={'level_0':'person_id'})
    return applicants

 

def generate_candidates_df(applicants, talents, sparta_day_results):
    applicants_and_talents = applicants.merge(talents, 
                                              left_on=['name', 'invite_date'], 
                                              right_on=['name', 'date'], 
                                              how='left')

    candidates = applicants_and_talents.merge(sparta_day_results, 
                                              left_on=['name', 'invite_date'], 
                                              right_on=['name', 'date'], 
                                              how='left')

    candidates = candidates.drop(candidates.index[candidates.person_id.duplicated().tolist()].tolist())
    candidates.reset_index(drop=True, inplace=True)
    candidates['person_id'] = candidates.index + 1
    return candidates

In [59]:
def address_id_in_candidates(candidates, address_table):
    merged = pd.merge(candidates, address_table, on='address')
    merged = merged.drop(columns=['city_x', 'address', 'postcode_x', 'city_y', 'postcode_y', 'date_y'])
    merged = merged.rename(columns={"date_x": "date"})
    return merged

In [60]:
candidates = generate_candidates_df(get_person_ids(df_csv), df_json, df_txt)

In [61]:
candidates = address_id_in_candidates(candidates, address_table)

In [13]:
def generate_courses_table(dataframe):
    courses_df  = dataframe
    courses_df = pd.DataFrame().assign(course = courses_df['course'], date = courses_df['date'])
    courses_df['course_id'] = courses_df.groupby(['course']).ngroup()
    courses_df = courses_df[['course', 'course_id', 'date']].copy()
    courses_df.set_index('course_id', inplace=True)
    courses_df.groupby(['course','date'])
    courses_df = courses_df.drop_duplicates()
    courses_df = courses_df.reset_index()
    courses_df.index = courses_df.index + 1
    courses_df['course_id'] = courses_df.index
    courses_df = courses_df.rename(columns={"course": "course_name"})
    
    return courses_df

In [14]:
courses_table = generate_courses_table(df_academy_csv)

In [15]:
# makes a weakness table dataframe
def generate_weakness_df(df):
    weaknesses_list = list()
    
    # for each row in the dataframe add distinct weaknesses to a list
    for index, row in df.iterrows():
        for weakness in row["weaknesses"]:
            if weakness not in weaknesses_list:
                weaknesses_list.append(weakness)
                
    # turn weakneseses list into a dataframe and add a column for the weakness id            
    weakness_table = pd.DataFrame(weaknesses_list, columns=['weakness'])
    weakness_table.index = weakness_table.index + 1
    weakness_table['weakness_id'] = weakness_table.index
    return weakness_table          

In [16]:
weakness_df = generate_weakness_df(df_json)

In [17]:
# takes talent jsons, talent csv, weakness dataframe as arguments
def generate_weakness_junc_df(candidates, weakness_df):
    weakness_junc_list = list()
    
    # for each row in the merged dataframe add (id, weakness) to a new dataframe for each weakness in weaknesses list
    for index, row in candidates.iterrows():
        person_id = row['person_id']
        weaknesses = row['weaknesses']
        
        if type(weaknesses) == list:
            for weakness in weaknesses:
                weakness_junc_list.append([person_id, weakness])
    
    weakness_junc = pd.DataFrame(weakness_junc_list, columns=['person_id', 'weakness'])
    
    weakness_junc = pd.merge(weakness_junc, weakness_df, on='weakness')[['person_id', 'weakness_id']] 
        
    return weakness_junc      

In [18]:
weakness_junc_df = generate_weakness_junc_df(candidates, weakness_df)

In [19]:
def generate_strengths_df(df):
    # Create a new DataFrame for strengths column only
    column_names = ['name', 'strengths']
    strengths = df[column_names]

    # Initialize an empty list to store row data
    strengths_list = []

    # Iterate over all rows
    for index, row in strengths.iterrows():
        strengths_list_row = row['strengths']

        # Iterate over each strength in the list
        for strength in strengths_list_row:
            if strength not in strengths_list:
                # Append row data to the list
                strengths_list.append(strength)

    # Create a new DataFrame from the list
    strengths_df = pd.DataFrame({'strength': strengths_list})
    strengths_df.index = strengths_df.index + 1
    strengths_df['strength_id'] = strengths_df.index
    # Return the new DataFrame
    return strengths_df

In [20]:
strengths_df = generate_strengths_df(df_json)

In [21]:
# takes talent jsons, talent csv, weakness dataframe as arguments
def generate_strengths_junc_df(candidates, strengths_df):
    strengths_junc_list = list()
    
    for index, row in candidates.iterrows():
        person_id = row['person_id']
        strengths = row['strengths']
        
        if type(strengths) == list:
            for strength in strengths:
                strengths_junc_list.append([person_id, strength])
    
    strengths_junc = pd.DataFrame(strengths_junc_list, columns=['person_id', 'strength'])

    strengths_junc = pd.merge(strengths_junc, strengths_df, on='strength')[['person_id', 'strength_id']] 
        
    return strengths_junc

In [22]:
strengths_junc_df = generate_strengths_junc_df(candidates, strengths_df)

In [23]:
def generate_tss_df(df, candidates):
    #Create names lists
    names = []
    technologies = []
    self_scores = []

    #Loop through dataframe
    for index, row in df.iterrows():
        try:
            techs = list(row['tech_self_score'].keys())
            for tech in techs:
                names.append(row['name'].upper())
                technologies.append(tech)
                self_scores.append(row['tech_self_score'][tech])
        except AttributeError:
            continue

    tech_self_scores = pd.DataFrame({'name':names, 'technology':technologies, 'self_score': self_scores})
    merged = pd.merge(tech_self_scores, candidates, on='name')[['person_id', 'technology', 'self_score']] 
    
    return merged

In [24]:
tech_scores = generate_tss_df(df_json, candidates)

In [25]:
def generate_academy_table(df_academy_csv, courses_table, candidates):
    merged = pd.merge(df_academy_csv, courses_table, left_on='course', right_on='course_name')[['name', 'course_id']]
    academy_table = pd.merge(merged, candidates, on='name')[['person_id', 'course_id']] 
    return academy_table

In [42]:
academy_table = generate_academy_table(df_academy_csv, courses_table, candidates)

In [27]:
def generate_trainers_table(df):
    old_name = "Ely Kely"
    new_name = "Elly Kelly"
    df.loc[:, ["trainer"]] = df.loc[:, ["trainer"]].replace(old_name, new_name)
    trainers_course = df[["trainer", "course", "date"]]
    trainer_course = trainers_course.drop_duplicates(subset = ["date"], keep= "first")
    trainer_course['trainer_id'] = range(1, len(trainer_course) + 1)
    trainers = trainer_course.set_index('trainer_id')
    trainers_table = trainers.drop(['course', 'date'], axis=1).drop_duplicates()
    trainers = trainers.rename(columns={"course": "course_name"})
    trainers['trainer_id'] = trainers.index
    return trainers, trainers_table

In [28]:
trainers, trainers_table = generate_trainers_table(df_academy_csv)

C:\Users\Mankabir\AppData\Local\Temp\ipykernel_15640\2851188400.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trainer_course['trainer_id'] = range(1, len(trainer_course) + 1)


In [29]:
def generate_trainers_junc(df_trainers, df_courses):
    trainers_junc = pd.merge(df_trainers, df_courses, on='course_name')[['course_id', 'trainer_id']] 
    return trainers_junc

In [30]:
trainers_junc = generate_trainers_junc(trainers, courses_table)

In [31]:
def use(key: str):
    s3 = boto3.client('s3')
    obj = s3.get_object(Bucket='data-eng-228-final-project', Key=key)
    filetype = key[key.index('.'):]
    if filetype == '.csv':
        data = pd.read_csv(obj['Body'])
    elif filetype == '.json':
        data = json.load(obj['Body'])
    elif filetype == '.txt':
        data = obj['Body'].read().decode('utf-8')
    return data


def make_wide(table):
    academy_results_wide = table.pivot(index=['name', 'week_no'],
                                       columns='attribute',
                                       values='score')
    academy_results = academy_results_wide.reset_index()
    return academy_results


def make_upper(df, attribute: str='name'):
    return df[attribute].transform(lambda attr: attr.upper())


def transform_acares(keystrings: list):
    tables = []

    for keystring in keystrings:
        current_ar = use(keystring)
        current_ar['name'] = make_upper(current_ar, 'name')
        raw_attributes = current_ar.columns[2:]

        names = []
        week_numbers = []
        attributes = []
        scores = []

        for name in current_ar['name']:
            for attribute in raw_attributes:
                names.append(name)
                week_numbers.append(attribute[attribute.index('W') + 1:])
                attributes.append(attribute[:attribute.index('_')])
                scores.append(float(current_ar[current_ar['name'] == name][attribute].tolist()[0]))
        academy_results_long = pd.DataFrame({'name': names,
                                             'week_no': week_numbers,
                                             'attribute': attributes,
                                             'score': scores})
        tables.append(academy_results_long)
    return pd.concat(tables)

In [32]:
s3_client = boto3.client('s3')
files_dict = {'academy_csv': [], 'json': [], 'txt': [], 'csv': []}
academy_csvs_file_names = extract_file_type(s3_client, 'Academy', files_dict, 'csv')
df1 = transform_acares(academy_csvs_file_names)
academy_results = make_wide(df1)

In [33]:
def join_person_id_to_acares(candidates, academy_results):
    passed_candidates = candidates[(candidates.result == 'PASS').tolist()]
    passed_candidates

 

    academy_results = academy_results.merge(passed_candidates[['person_id', 'name']], left_on='name', right_on='name')
    return academy_results

In [40]:
academy_results = join_person_id_to_acares(candidates, academy_results)

In [41]:
display(academy_results)

,name,week_no,Analytic,Determined,Imaginative,Independent,Professional,Studious,person_id
0,ADAH SPENCERS,1,5.0,2.0,3.0,3.0,5.0,4.0,288
1,ADAH SPENCERS,2,4.0,1.0,1.0,5.0,1.0,4.0,288
2,ADAH SPENCERS,3,6.0,1.0,2.0,3.0,7.0,2.0,288
3,ADAH SPENCERS,4,4.0,8.0,5.0,6.0,7.0,3.0,288
4,ADAH SPENCERS,5,2.0,8.0,5.0,5.0,3.0,3.0,288
...,...,...,...,...,...,...,...,...,...
3261,ZOLLIE DANKS,4,8.0,3.0,5.0,4.0,6.0,6.0,4579
3262,ZOLLIE DANKS,5,5.0,5.0,8.0,4.0,7.0,7.0,4579
3263,ZOLLIE DANKS,6,7.0,7.0,7.0,8.0,7.0,4.0,4579
3264,ZOLLIE DANKS,7,4.0,8.0,6.0,8.0,8.0,8.0,4579


In [35]:
#takes in a df and import its to table in a databse .db file
def df_to_db(df: pd.DataFrame, table_name: str, database_name: str):
    con = sqlite3.connect(database_name)
    df.to_sql(table_name, con, if_exists='append', index=False)
    con.close()

In [36]:
def all_df_to_db(df_list: [], table_names:[], database_name:str):
    for df, table_name in zip(df_list, table_names):
        df_to_db(df, table_name, database_name)

In [86]:
df_list = [address_table, candidates, courses_table, weakness_df, weakness_junc_df, strengths_df, strengths_junc_df, tech_scores, academy_table, trainers_table, trainers_junc, academy_results]
table_list =['address', 'candidates', 'courses', 'weakness', 'weakness_junc', 'strength', 'strength_junc', 'tech_scores', 'academy', 'trainers', 'trainers_junc', 'academy_results']

In [99]:
# Create a SQLite connection
conn = sqlite3.connect('my_database.db')
c = conn.cursor()

# Enable foreign key constraint
c.execute("PRAGMA foreign_keys = ON;")

# Define the schema as a list of strings
schemas = [
    '''
    CREATE TABLE address (
        address_id INTEGER PRIMARY KEY,
        address TEXT,
        city TEXT,
        postcode TEXT
    )
    ''',
    '''
    CREATE TABLE candidates (
        person_id INTEGER PRIMARY KEY,
        name TEXT,
        invite_date TEXT,
        invited_by TEXT,
        address_id INTEGER,
        gender TEXT,
        dob TEXT,
        email TEXT,
        phone_number TEXT,
        uni TEXT,
        degree INTEGER,
        self_development INTEGER,
        result TEXT,
        geo_flex INTEGER,
        financial_support_self INTEGER,
        course_interest TEXT,
        sparta_day_date TEXT,
        psychometrics_score INTEGER,
        presentation_score INTEGER,
        sparta_day_location TEXT,
        date TEXT,
        academy TEXT,
        FOREIGN KEY(address_id) REFERENCES address(address_id)
    )
    ''',
    '''
    CREATE TABLE tech_scores (
        person_id INTEGER,
        tech_scores_id INTEGER,
        scores INTEGER,
        PRIMARY KEY (person_id, tech_scores_id),
        FOREIGN KEY(person_id) REFERENCES candidates(person_id)
    )
    ''',
    '''
    CREATE TABLE weakness (
        weakness_id INTEGER PRIMARY KEY,
        weakness TEXT
    )
    ''',
    '''
    CREATE TABLE weakness_junc (
        person_id INTEGER,
        weakness_id INTEGER,
        PRIMARY KEY (person_id, weakness_id),
        FOREIGN KEY(person_id) REFERENCES candidates(person_id),
        FOREIGN KEY(weakness_id) REFERENCES weakness(weakness_id)
    )
    ''',
    '''
    CREATE TABLE strength (
        strength_id INTEGER PRIMARY KEY,
        strength TEXT
    )
    ''',
    '''
    CREATE TABLE strength_junc (
        person_id INTEGER,
        strength_id INTEGER,
        PRIMARY KEY (person_id, strength_id),
        FOREIGN KEY(person_id) REFERENCES candidates(person_id),
        FOREIGN KEY(strength_id) REFERENCES strength(strength_id)
    )
    ''',
    '''
    CREATE TABLE academy_results (
        person_id INTEGER,
        week_no INTEGER,
        analytic INTEGER,
        independant INTEGER,
        determined INTEGER,
        professional INTEGER,
        imaginative INTEGER,
        PRIMARY KEY (person_id, week_no),
        FOREIGN KEY(person_id) REFERENCES candidates(person_id)
    )
    ''',
    '''
    CREATE TABLE courses (
        course_id INTEGER PRIMARY KEY,
        course_name TEXT,
        date TEXT
    )
    ''',
    '''
    CREATE TABLE trainers (
        trainer_id INTEGER PRIMARY KEY,
        trainer_name TEXT
    )
    ''',
    '''
    CREATE TABLE trainers_junc (
        course_id INTEGER,
        trainer_id INTEGER,
        PRIMARY KEY (course_id, trainer_id),
        FOREIGN KEY(course_id) REFERENCES courses(course_id),
        FOREIGN KEY(trainer_id) REFERENCES Trainers(trainer_id)
    )
    ''',
    '''
    CREATE TABLE academy (
        person_id INTEGER,
        course_id INTEGER,
        PRIMARY KEY (person_id, course_id),
        FOREIGN KEY(person_id) REFERENCES candidates(person_id),
        FOREIGN KEY(course_id) REFERENCES courses(course_id)
    )
    '''
]

# Execute each schema
for schema in schemas:
    c.execute(schema)

# Commit the changes and close the connection
conn.commit()
conn.close()


In [104]:
all_df_to_db(df_list, table_list, 'my_database24.db')

In [103]:
display(candidates)

,person_id,name,gender,dob,email,phone_number,uni,degree,invited_by,invite_date,date,self_development,geo_flex,financial_support_self,result,course_interest,psychometrics_score,presentation_score,academy,address_id
0,1,ESME TRUSSLOVE,F,1994-08-04,etrusslove0@google.es,442957830228,"Saint George's Hospital Medical School, Univer...",2.1,BRUNO BELLBROOK,2019-04-10,NaT,NaN,NaN,NaN,NaN,NaN,39.0,23.0,London Academy,1
1,2,MATTHAEUS AUDAS,M,NaT,maudas1@mapquest.com,449577280155,Keele University,2.1,DORIS BELLASIS,2019-04-30,2019-04-30,YES,YES,YES,PASS,DATA,59.0,20.0,London Academy,2
2,3,CHEREY TOLLFREE,F,1992-12-08,ctollfree2@netvibes.com,445887496002,"King's College London, University of London",2.1,GISMO TILLING,2019-04-25,2019-04-25,YES,YES,YES,PASS,BUSINESS,59.0,18.0,Birmingham Academy,3
3,4,ERYN SPEERS,F,NaT,espeers3@shinystat.com,441487870613,University of Edinburgh,2.1,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,5,THEADORA BERKELAY,F,1995-11-03,tberkelay4@godaddy.com,448414683619,University of Leicester,2.1,STACEY BROAD,2019-04-02,NaT,NaN,NaN,NaN,NaN,NaN,59.0,12.0,London Academy,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4698,4687,CLYVE GILLHESPY,M,1995-12-15,cgillhespybj@buzzfeed.com,449043432218,University of Liverpool,2.1,BRUNO BELLBROOK,2019-09-26,2019-09-26,YES,YES,YES,PASS,BUSINESS,59.0,24.0,Birmingham Academy,4594
4699,4688,VACLAV PIETESCH,M,1994-11-09,vpieteschbk@mac.com,444556316125,Sheffield Hallam University,2.2,STACEY BROAD,2019-09-12,2019-09-12,YES,YES,NO,FAIL,ENGINEERING,61.0,27.0,Birmingham Academy,4595
4700,4689,KASSI LUCIO,F,1994-04-24,kluciobl@exblog.jp,448343429323,University of Buckingham,2.1,FIFI ETON,2019-09-03,NaT,NaN,NaN,NaN,NaN,NaN,60.0,24.0,London Academy,4596
4701,4690,VIVIANNA LETTY,F,NaT,vlettybm@google.com.hk,445347583140,Leeds Metropolitan University,1.0,BRUNO BELLBROOK,2019-09-19,NaT,NaN,NaN,NaN,NaN,NaN,56.0,19.0,Birmingham Academy,4597


In [102]:
candidates = candidates.drop_duplicates(subset='person_id', keep='first')

In [107]:
# Create a SQLite connection
conn = sqlite3.connect('my_database24.db')

person_id = 24  # change this to the specific name_id you're interested in

# SQLite query string
query = '''
SELECT *
FROM candidates
LEFT JOIN academy ON candidates.person_id = academy.person_id
LEFT JOIN academy_results ON candidates.person_id = academy_results.person_id
LEFT JOIN strength_junc ON candidates.person_id = strength_junc.person_id
LEFT JOIN weakness_junc ON candidates.person_id = weakness_junc.person_id
LEFT JOIN tech_scores ON candidates.person_id = tech_scores.person_id
WHERE candidates.person_id = ?
'''


# Execute the query
df_query = pd.read_sql_query(query, conn, params=(person_id,))


conn.close()


In [110]:
df_query

,person_id,name,gender,dob,email,phone_number,uni,degree,invited_by,invite_date,...,Professional,Studious,person_id,person_id,strength_id,person_id,weakness_id,person_id,technology,self_score
0,24,RIP MCILVENNY,M,1999-05-31 00:00:00,rmcilvennyn@tripod.com,443402647575,University of Hertfordshire,1.0,GISMO TILLING,2019-04-18 00:00:00,...,None,None,None,24,13,24,15,24,R,2
1,24,RIP MCILVENNY,M,1999-05-31 00:00:00,rmcilvennyn@tripod.com,443402647575,University of Hertfordshire,1.0,GISMO TILLING,2019-04-18 00:00:00,...,None,None,None,24,13,24,15,24,SPSS,3
2,24,RIP MCILVENNY,M,1999-05-31 00:00:00,rmcilvennyn@tripod.com,443402647575,University of Hertfordshire,1.0,GISMO TILLING,2019-04-18 00:00:00,...,None,None,None,24,17,24,15,24,R,2
3,24,RIP MCILVENNY,M,1999-05-31 00:00:00,rmcilvennyn@tripod.com,443402647575,University of Hertfordshire,1.0,GISMO TILLING,2019-04-18 00:00:00,...,None,None,None,24,17,24,15,24,SPSS,3
